# OpenFisca-UK

This book contains an introduction to using OpenFisca-UK to model UK taxes and benefits. It is currently a work in progress and may be added to. OpenFisca-UK is a microsimulation model of the UK tax and benefit system: it is a model which calculates variable values over UK entities from given policy parameters and structures. In practice, this gives it two main uses: calculating statistics under current tax and benefit law, and simulating effects of potential new changes to the legislation.

We're grateful to the [UKMOD](https://www.iser.essex.ac.uk/research/projects/ukmod) team for publishing descriptions of their model; our ability to reference these descriptions accelerated OpenFisca UK's development. UKMOD is maintained, developed and managed by the Centre for Microsimulation and Policy Analysis at the Institute for Social and Economic Research (ISER), University of Essex.

Code examples and outputs are re-run automatically on each new version of OpenFisca-UK.

## Short demo

### Baseline estimation

Calculating, for example, the total tax (Income Tax and National Insurance) liability by region can be done with the following code:

In [3]:
from openfisca_uk import Microsimulation
import pandas as pd

sim = Microsimulation()

df = sim.df(["national_insurance", "income_tax", "people"])

summary = pd.DataFrame(
    df.groupby(sim.calc("region", map_to="person")).sum()
).sort_values(by="people", ascending=False)
summary.national_insurance = summary.national_insurance.apply(
    lambda x: round(x / 1e9, 1)
)
summary.income_tax = summary.income_tax.apply(lambda x: round(x / 1e9, 1))
summary.people = summary.people.apply(lambda x: round(x / 1e6, 1))
summary.columns = [
    "National Insurance (£bn)",
    "Income Tax (£bn)",
    "Population (millions)",
]
summary

,National Insurance (£bn),Income Tax (£bn),Population (millions)
SOUTH_EAST,11.0,34.3,9.0
LONDON,12.0,37.3,9.0
NORTH_WEST,6.6,14.0,7.2
EAST_OF_ENGLAND,6.7,17.6,6.2
WEST_MIDLANDS,5.1,12.5,5.8
SOUTH_WEST,5.4,15.0,5.5
YORKSHIRE,4.3,9.6,5.4
WALES,5.4,12.2,5.4
EAST_MIDLANDS,4.2,9.4,4.7
SCOTLAND,2.4,5.4,3.1


### Reform evaluation

Below is an example of simulating the effects of a reform (namely, increasing the basic rate of income tax from 20% to 23%).

In [4]:
from openfisca_uk.api import *


def change_tax_parameters(parameters):
    parameters.tax.income_tax.rates.uk.brackets[0].rate.update(
        period=periods.period("year:2019:10"), value=0.23
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform)
revenue = reformed.calc("tax").sum() - baseline.calc("tax").sum()
f"Revenue: £{round(revenue / 1e+9, 1)}bn"

'Revenue: £14.7bn'